In [ ]:

public class Item // дополнительный класс
{
    public string Name { get; set; }
    public decimal Price { get; set; }

    public Item(string name, decimal price)
    {
        Name = name;
        Price = price;
    }
}

public class Order // базовый класс
{
    private readonly IItemService _itemService; //зависимость

    public int OrderId { get; set; }
    public DateTime CreationDate { get; set; }
    public decimal TotalAmount { get; set; }
    protected List<Item> Items { get; set; } // позволяет хранить упорядоченные элементы

    public Order(int orderId, IItemService itemService)
    {
        OrderId = orderId;
        CreationDate = DateTime.Now;
        Items = new List<Item>();
        TotalAmount = 0;
        _itemService = itemService;
    }

    public virtual void CalculateTotal()
    {
        TotalAmount = 0;
        foreach (var item in Items)
        {
            TotalAmount += item.Price;
        }
    }

    public virtual void AddItem(Item item)
    {
        _itemService.AddItem(this, item); //использование зависимости 
        //позволяет классу Order использовать реализацию IItemService для добавления и удаления элементов, не зная конкретной реализации
        CalculateTotal();
    }

    public virtual void AddItem(string name, decimal price) // перегрузка
    {
        Item newItem = new Item(name, price);
        AddItem(newItem);
    }

    public virtual void RemoveItem(Item item)
    {
        _itemService.RemoveItem(this, item); //использование зависимости
        CalculateTotal();
    }

    public void AddItemDirectly(Item item)
    {
        Items.Add(item);
    }

    public void RemoveItemDirectly(Item item)
    {
        Items.Remove(item);
    }

    public List<Item> GetItems()
    {
        return Items.ToList(); 
    }

    public virtual void RemoveItem(string itemName)
    {
        var itemToRemove = Items.FirstOrDefault(i => i.Name == itemName);
        if (itemToRemove != null)
        {
            RemoveItem(itemToRemove);
        }
    }
    public virtual void PrintDetails()
    {
        Console.WriteLine($"Заказ ID: {OrderId}, Дата создания: {CreationDate}, Общая сумма: {TotalAmount} р.");
    }
}


public interface IItemService // интерфейс зависимости
{
    void AddItem(Order order, Item item);
    void RemoveItem(Order order, Item item);
}

public class ItemService : IItemService //зависимость содержит логику для + - товара
{
    public void AddItem(Order order, Item item)
    {
        order.AddItemDirectly(item); // внедрение зависимости
    }

    public void RemoveItem(Order order, Item item)
    {
        order.RemoveItemDirectly(item); // внедрение зависимости
    }
}

public class OnlineOrder : Order // простое наследование
{
    public string CustomerEmail { get; set; }

    public OnlineOrder(int orderId, string customerEmail, IItemService itemService) : base(orderId, itemService) //исп завис
    {
        CustomerEmail = customerEmail;
    }

    public override void AddItem(Item item)
    {
        base.AddItem(item);
        PrintAdd(item);
    }

    public override void RemoveItem(Item item)
    {
        base.RemoveItem(item);
        PrintRemove(item);
    }

    public virtual void PrintAdd(Item item)
    {
        Console.WriteLine($"Товар '{item.Name}' добавлен в онлайн-заказ для {CustomerEmail}");
    }

    public virtual void PrintRemove(Item item)
    {
        Console.WriteLine($"Товар '{item.Name}' удален из онлайн-заказа.");
    }

    public override void PrintDetails()
    {
        base.PrintDetails();
        Console.WriteLine($"Email клиента: {CustomerEmail}");
    }
}

public class PhysicalOrder : OnlineOrder // сложное наследование
{
    public string DeliveryAddress { get; set; }

    public PhysicalOrder(int orderId, string deliveryAddress, IItemService itemService) : base(orderId, "---", itemService) //исп завис
    {
        DeliveryAddress = deliveryAddress;
    }

    public override void PrintAdd(Item item) //перекрытие
    {
        Console.WriteLine($"Товар '{item.Name}' добавлен в физический заказ.");
    }

    public override void PrintRemove(Item item) //перекрытие
    {
        Console.WriteLine($"Товар '{item.Name}' удален из физического заказа. Проверьте возврат на адресе {DeliveryAddress}");
    }

    public override void PrintDetails() //перекрытие
    {
        base.PrintDetails();
        Console.WriteLine($"Адрес доставки: {DeliveryAddress}");
    }
}

public interface ISpecializedOrder // интерфейс
{
    void SetSpecialConditions(string specialConditions);
    void Sale(Item item);

}

public class SpecializedOrder : Order, ISpecializedOrder // множественное наследование
{
    public string SpecialConditions { get; private set; }
    public bool HasSpecialConditions { get; private set; } // булева переменная

    public SpecializedOrder(int orderId, string specialConditions, IItemService itemService) : base(orderId, itemService) //исп завис
    {
        ((ISpecializedOrder)this).SetSpecialConditions(specialConditions);
    }

   
    void ISpecializedOrder.SetSpecialConditions(string specialConditions) //явная реализация интерфейса тк есть название интерфейса
    {
        SpecialConditions = specialConditions;
        HasSpecialConditions = !string.IsNullOrEmpty(specialConditions); // проверка на пустоту или null
    }

    
    void ISpecializedOrder.Sale(Item item) // явная реализация интерфейса
    {
        Console.WriteLine($"На товар '{item.Name}' действует скидка 10%!");
    }

    public override void CalculateTotal()
    {
        base.CalculateTotal();
        if (HasSpecialConditions)
        {
            TotalAmount *= 0.9m;  
        }
    }

    public override void PrintDetails()
    {
        base.PrintDetails();
        if (HasSpecialConditions)
        {
            Console.WriteLine($"Специальные условия: {SpecialConditions}");
        }
        else
        {
            Console.WriteLine("Специальные условия отсутствуют.");
        }
    }
}


IItemService itemService = new ItemService();

Item item1 = new Item("Книга", 150);
Item item2 = new Item("Игрушка", 200);
Item item3 = new Item("Подушка", 700);

OnlineOrder onlineOrder = new OnlineOrder(1, "pochta@mail.ru", itemService);
onlineOrder.AddItem(item1);
onlineOrder.PrintDetails();
Console.WriteLine();

PhysicalOrder physicalOrder = new PhysicalOrder(2, "ул. Мельникайте, д. 70, г. Тюмень", itemService);
physicalOrder.AddItem(item2);
physicalOrder.RemoveItem(item2);
physicalOrder.PrintDetails();
Console.WriteLine();

SpecializedOrder specializedOrder = new SpecializedOrder(3, "Скидка на новый продукт", itemService);
ISpecializedOrder specializedOrderInterface = specializedOrder; // объект типа интерфейса
specializedOrderInterface.SetSpecialConditions("Скидка на новый продукт"); //явн реал
specializedOrderInterface.Sale(item3); //явн реал
specializedOrder.AddItem(item3);
specializedOrder.CalculateTotal();
specializedOrder.PrintDetails();

Товар 'Книга' добавлен в онлайн-заказ для pochta@mail.ru
Заказ ID: 1, Дата создания: 11/23/2024 11:59:53 AM, Общая сумма: 150 р.
Email клиента: pochta@mail.ru

Товар 'Игрушка' добавлен в физический заказ.
Товар 'Игрушка' удален из физического заказа. Проверьте возврат на адресе ул. Мельникайте, д. 70, г. Тюмень
Заказ ID: 2, Дата создания: 11/23/2024 11:59:53 AM, Общая сумма: 0 р.
Email клиента: ---
Адрес доставки: ул. Мельникайте, д. 70, г. Тюмень

На товар 'Подушка' действует скидка 10%!
Заказ ID: 3, Дата создания: 11/23/2024 11:59:53 AM, Общая сумма: 630.0 р.
Специальные условия: Скидка на новый продукт


<h2 style="color:DodgerBlue">Описание проекта:</h2>

----

Описание задачи:

Создать базовый класс Item в C#, который будет представлять информацию о
товарах, которые могут быть заказаны или возвращены. На основе этого класса
разработать 2-3 производных класса, демонстрирующих принципы наследования и
полиморфизма. В каждом из классов должны быть реализованы новые атрибуты и
методы, а также переопределены некоторые методы базового класса для
демонстрации полиморфизма.

Требования к базовому классу Item:

• Атрибуты: ID товара (ItemId), Название (Name), Цена (Price).

• Методы:

o GetDetails(): метод для получения детальной информации о товаре.

o CalculateDiscount(): метод для расчета скидки на товар.

o ApplyDiscount(decimal discount): метод для применения скидки к цене
товара.

#### Дополнительное задание